In [ ]:
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
import os
import dotenv

dotenv.load_dotenv()

In [ ]:
# Werte zur Eingabe für das ganze Dokument

# latitude = 51.1657
# longitude = 10.4515
# start_date = "2023-01-01"
# end_date = "2023-12-31"


True

In [14]:
df_dwd = pd.read_csv('/Users/tomkurze/Stackfuel DPP /Stackfuel-DPP/data/raw/Balkonkraftwerk/produkt_st_stunde_20050101_20251231_01975.txt',
                 sep=';',
                 parse_dates=['MESS_DATUM','MESS_DATUM_WOZ'],
                 date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d%H:%M"))
display(df.head, df.dtypes)

/var/folders/c5/491yhz3s1y93gvqdkz7t62p00000gn/T/ipykernel_37260/2147556409.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_dwd = pd.read_csv('/Users/tomkurze/Stackfuel DPP /Stackfuel-DPP/data/raw/Balkonkraftwerk/produkt_st_stunde_20050101_20251231_01975.txt',


<bound method NDFrame.head of       mRID          Startzeit            Endzeit Resolution Position  Preis
0        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        1  22.40
1        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        2  -7.40
2        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        3 -10.16
3        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        4  -5.30
4        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        5   5.54
...    ...                ...                ...        ...      ...    ...
50039  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       90   9.61
50040  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       91   4.50
50041  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       92   0.10
50042  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       93   1.14
50043  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       94   0.00

[50044 rows x 6 columns]>

mRID           object
Startzeit      object
Endzeit        object
Resolution     object
Position       object
Preis         float64
dtype: object

In [ ]:
from entsoe import EntsoePandasClient

client = EntsoePandasClient(api_key=os.getenv('TOKEN'))

start = pd.Timestamp('20250101', tz='Europe/Brussels')
end = pd.Timestamp('20260101', tz='Europe/Brussels')
country_code = 'DE_LU'  # Belgium
country_code_from = 'DE_LU'  # France
country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A44'
resolution = '15min'
sequence = 2

# methods that return Pandas Series
price_series = client.query_day_ahead_prices_local(country_code=country_code,
                                    sequence=2,
                                    start=start,
                                    end=end,
                                    resolution='15min')

# Umwandeln in einen DF mit entsprechender Spaltenbezeichnung
df_prices = price_series.reset_index()
df_prices = df_new.rename(columns={'index':'Datum', 0:'Preis'})
df_prices['Datum'] = df_prices['Datum'].dt.tz_localize(None).dt.tz_localize('UTC')
print(df_prices.dtypes)
print(df_prices)

In [45]:
# Abruf der relevanten Daten für die Sonneneinstrahlung bei NASA POWER

website_url_meteo = "https://archive-api.open-meteo.com/v1/archive"

# Parameter für den API-Aufruf
params = {
	"latitude": 53.645,
	"longitude": 10.04,
	"start_date": "2025-01-01",
	"end_date": "2025-12-31",
	"hourly": ["shortwave_radiation", "diffuse_radiation", "direct_normal_irradiance"],
}

# API-Anfrage senden
response = requests.get(website_url_meteo, params=params)

# Statuscode der Antwort ausgeben
print("Statuscode:", response.status_code)

# Antwort als JSON in Pandas DF ausgeben (falls erfolgreich)
if response.status_code == 200:
    data = response.json()
    parameter = data['hourly']
    df_meteo = pd.DataFrame({
		'Datum':pd.to_datetime(parameter['time']),
        'Global_Radiation':parameter['shortwave_radiation'],
        'Direct_Radiation':parameter['direct_normal_irradiance'],
        'Diffuse_Radiation':parameter['diffuse_radiation']
	})

    print('Daten erfolgreich im df_meteo gespeichert.')

else:
    print("Fehler beim Abrufen der Daten. Statuscode:", response.status_code)
    print("Fehlertext:", response.text)

Statuscode: 200
Daten erfolgreich im df_meteo gespeichert.


In [ ]:
df_meteo

,Datum,Global_Radiation,Direct_Radiation,Diffuse_Radiation
count,8760,8760.000000,8760.000000,8760.000000
mean,2025-07-02 11:30:00,133.375228,164.089680,52.349315
min,2025-01-01 00:00:00,0.000000,0.000000,0.000000
25%,2025-04-02 05:45:00,0.000000,0.000000,0.000000
50%,2025-07-02 11:30:00,6.000000,0.000000,5.000000
75%,2025-10-01 17:15:00,207.000000,279.675000,89.000000
max,2025-12-31 23:00:00,859.000000,903.200000,365.000000
std,NaN,204.000422,255.763495,73.354433
